In [ ]:
# This notebook extracts LLhs from fit files for the 1D distributions
# Note that due to some small historical ugliness we have two formats here.
#   arising because Amp0 was re-run more elegantly later.


In [ ]:
import numpy as np 
import glob
import os
import re


In [ ]:
def RunIt(outpath, params):
    outfile = open(outpath, "w")
    count=0
    PrintEvery=100
    fit_dirs = []
    for param in params:    
        fit_dirs = glob.glob(rootdir + "fit." + str(param) +'*')
        print("Searching for files: " + rootdir + "fit." + str(param) +'*')
        if not fit_dirs :
            print("WARNING: No Fit Directories Found For " + str(param) + " In " + str(rootdir) )
            continue
        else:
            print("Found "  + str(len(fit_dirs)) + " Fit Directories....")

 
        print("Writing Output to: "  + str(outfile))

        for fit_dir in fit_dirs:
            try:
                fit_file = open(str(fit_dir) + "/out",'r')
            except:
                print("WARNING: " + str(fit_dir) + "/out Does Not Exist!" )
            else:
                try:
                  #  lines=fit_file.readlines()[2]
                    lines= fit_file.readlines()
#                    llh_string = re.findall("LLH=\d+\.\d+", lines[2])
                    for line in lines:
                        if 'LLH' in line:
                            LLH=re.findall("LLH=\d+\.\d+", line)[0][4:]
                            DT=line.split("=")[2][0:-3]

                except:
                    print("WARNING: " + str(fit_file) + " Is Empty!" )
                else:
                    try:
                        shift = float(re.findall(r".\dp\d\d\d", str(fit_dir))[0].replace("p", "."))
                    except:
                        try:
                            shift = float(re.findall(r".\dp\d\d", str(fit_dir))[0].replace("p", "."))
                        except:
                            try:
                                shift = float(re.findall(r".\dp\d", str(fit_dir))[0].replace("p", "."))
                            except:
                                if "_0.dat" in str(fit_dir):
                                    shift = 0
                                else:
                                    print("WARNING: Unable to resolve shift. Setting shift to nan.")
                                    shift = float('NaN')

                    dom = fit_dir.rsplit('dat-', 1)[1]
                    
                    fit_file.close()
                    outfile.write(param.split("_")[0]+ " " + param.split("_")[1]+ " " +str(shift) + " " + str(LLH) + " " + str(dom) +" "  + str(DT)+ "\n")


                    count=count+1
                    if(count%PrintEvery==0):
                        print("Absorbed "+str(count) + " LLHs")
                   # print(str(shift) + " " + str(llh) + " " + str(dom))
    outfile.close()


    print("Done!")





In [ ]:
rootdir = '/data/ana/NuFSGenMC/MultiSim/Fits/Fits/Phs/'
phsparams=[]
for i in range(1,10):
    phsparams.append("Phs_"+str(i))
print phsparams
phsoutpath="/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/LLHs/Phs.txt"

In [ ]:
RunIt(phsoutpath,phsparams)

In [ ]:
rootdir = '/data/ana/NuFSGenMC/MultiSim/Fits/Fits/Amp/'
ampparams=[]
for i in range(1,10):
    ampparams.append("Amp_"+str(i))
print ampparams
ampoutpath="/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/LLHs/Amp.txt"

In [ ]:
RunIt(ampoutpath,ampparams)

In [ ]:
def RunItNewFormat(rootdir, outpath):
    count=0
    PrintEvery=100
    fit_dirs = []
    fit_files = glob.glob(rootdir + "/out*")
    if not fit_files :
        print("WARNING: No Fit Directories Found In " + str(rootdir) )
        return
    else:
        print("Found "  + str(len(fit_files)) + " Fit Files....")


    outfile = open(outpath, "a")

    print("Writing Output to: "  + str(outfile))

    for fit_filen in fit_files:
        try:
            fit_file = open(str(fit_filen),'r')
        except:
            print("WARNING: " + str(fit_filen) + " Does Not Exist!" )
        else:
            try:
              #  lines=fit_file.readlines()[2]
                lines= fit_file.readlines()     
            except:
                print("WARNING: " + str(fit_filen) + " Is Empty!" )
                break
            HeaderLine=lines[2]
            Param=HeaderLine.split("_")[0]
            ParamNum=HeaderLine.split("_")[1].split("+")[0].split("-")[0]
         #   print HeaderLine
            try:
                shift = float(re.findall(r".\dp\d\d\d", str(HeaderLine))[0].replace("p", "."))
            except:
                try:
                    shift = float(re.findall(r".\dp\d\d", str(HeaderLine))[0].replace("p", "."))
                except:
                    try:
                        shift = float(re.findall(r".\dp\d", str(HeaderLine))[0].replace("p", "."))
                    except:
                        if "_0.dat" in str(HeaderLine):
                            shift = 0
                        else:
                            print("WARNING: Unable to resolve shift. Setting shift to nan.")
                            shift = float('NaN')
            

            for line in lines[3:-1]:
                if "DOM " in line:
                    if not "Bad" in line:
                     #   print line.split()
                        DOM=int(line.split()[1])
                        
                if 'LLH' in line:
                    LLH=re.findall("LLH=\d+\.\d+", line)[0][4:]
                    DT=line.split("=")[2][0:-3]
                    if(DOM!=3 and DOM!=1 and DOM!=21):
                        writeline=str(Param)+ " "+ str(ParamNum)+" " +str(shift) + " " + str(LLH) + " " + str(DOM) + " " + str(DT)
                #    print writeline
                #    print line
                        outfile.write(writeline + "\n")                    

                    count=count+1
                    if(count%PrintEvery==0):
                        print("Absorbed "+str(count) + " LLHs")
        fit_file.close()
    outfile.close()

    print("Done!")






In [ ]:
RunItNewFormat("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/ConstraintJobsOut/Amp0", "/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/LLHs/Amp0.txt")